# Goal: 

We aim to develop a `Question-Answering (QA) system` that retrieves and processes data from a `Dataverse`, leveraging models from `Models.corp` and an `in-memory Milvus Vector DB` for efficient indexing and retrieval.

### Objective:

To build an efficient `Retrieval-Augmented Generation (RAG)` system that pulls data from a `Dataverse`, extracts relevant chunks of text, generates embeddings using a pre-trained transformer model, and answers user queries through an LLM. The system will optimize `document chunking, similarity matching, and querying` to provide accurate and context



### Prerequisites

1. Make sure you are on RH VPN (Some links and services below require internal / VPN access.)
2. LangChain - https://github.com/langchain-ai/langchain
3. List of hosted/managed LLMs (Models.corp) - https://gitlab.cee.redhat.com/models-corp/user-documentation/-/blob/main/README.md
4. Granite 3.1 Model (models.corp) - https://granite-3-1-8b-instruct--apicast-production.apps.int.stc.ai.prod.us-east-1.aws.paas.redhat.com/v1
    * Model details available at https://gitlab.cee.redhat.com/models-corp/user-documentation/-/blob/main/models/granite-3-1-8b-instruct.md
5. In memory vectordb/Milvus - https://python.langchain.com/docs/integrations/vectorstores/milvus/
6. Embedding Model (mixedbread-ai/mxbai-embed-large-v1) - https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1PubMedQA_instruction 
7. Getting access to a dataset in Dataverse (send an email to dataverse-access-request@redhat.com. You can get access to data sets based on your Red Hat role and project needs.)
8. Get access to MOSAIC sandbox environment - https://redhat.service-now.com/help?id=sc_cat_item&sys_id=685b8cf987c74a5079f021b2debb353a

### Key Components:

1. **Connection to Dataverse:**:  

   Fetches structured and unstructured data from Dataverse for processing.

2. **RecursiveCharacterTextSplitter:**:  

   Splits documents into smaller chunks for easier processing.

3. **Milvus**:  

   Stores document embeddings as vectors for efficient retrieval.

4. **HuggingFaceEmbeddings**:  

   Generates numerical embeddings from text chunks for similarity-based retrieval.

5. **RAG**:  

   Uses document retrieval and LLMs to generate context-aware responses to queries.

### Integration & Security:
- **Environment Variables**: 
  - Use a `.env` file to store and access API keys locally.
  - Make sure not to expose or share this file!
  
- **External APIs**: 
  - Utilize model and embedding APIs for seamless interaction between the components of the RAG system.

### Expected Outcome:
- A robust pipeline capable of:
  - **Extracting** Data from Dataverse.
  - **Generating** high-quality embeddings for efficient retrieval.
  - **Querying** with contextually relevant answers using RAG.
  - Providing accurate and consistent responses to user queries in a scalable and secure manner.

# Install dependencies
Uncomment the following cell and install the dependencies.

In [ ]:
# pip install snowflake-connector-python pandas langchain_community langchain_text_splitters beautifulsoup4 pymilvus langchain_milvus langchain_huggingface huggingface_hub langchain_openai requests python-dotenv

## Imports & Dependencies

This section includes all necessary libraries and modules required for data processing, embedding generation, retrieval, and querying using the RAG pipeline.


In [ ]:
import snowflake.connector
import pandas as pd
import json
import os
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
from dotenv import load_dotenv
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import warnings
warnings.filterwarnings('ignore')

# Building Connection and Fetching Content from Dataverse

This script connects to Dataverse(Snowflake) using `conn_params`, retrieves the `CONTENT` column from the `USER_GUIDES` table, and extracts JSON content for processing. 

**Note:** Load the user and role parameters from environment variables.


In [ ]:
load_dotenv()

# Snowflake connection parameters
conn_params = {
    'account': 'GDADCLC-RHSANDBOX',
    'user': os.getenv('SF_USER'), 
    'authenticator': 'externalbrowser', 
    'warehouse': 'aipoc_group_xs_wh',
    'database': 'AIPOC_DB', 
    'schema': 'MARTS_RHSC_USER_GUIDES', 
    'role': os.getenv('SF_ROLE'), 
}

def fetch_documents():
    """Fetch CONTENT and NAME from Snowflake and return as LangChain Document objects."""
    conn = snowflake.connector.connect(**conn_params)
    cursor = conn.cursor()

    # Fetch document content and name
    query = "SELECT NAME, CONTENT FROM AIPOC_DB.MARTS_RHSC_USER_GUIDES.USER_GUIDES"
    cursor.execute(query)

    df = pd.DataFrame(cursor.fetchall(), columns=['NAME', 'CONTENT'])

    cursor.close()
    conn.close()

    # Convert JSON variant to readable text
    def extract_text(content):
        try:
            parsed_json = json.loads(content)  
            return parsed_json.get("content", "No content available")  
        except json.JSONDecodeError:
            return content  

    df['CONTENT'] = df['CONTENT'].apply(extract_text)

    
    documents = [
        Document(page_content=text, metadata={"name": name}) 
        for name, text in zip(df['NAME'], df['CONTENT'])
    ]
    
    return documents

# Fetch list of Documents first after reading from snowflake table
documents = fetch_documents()


/opt/anaconda3/envs/Snowflake/lib/python3.9/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.redhat.com/auth/realms/EmployeeIDP/protocol/saml?SAMLRequest=lZJfb9owFMW%2FSuQ9EzuQsdYCKgplQ6IdIqHa9uYkF2LVsVNfp4FvP4c%2FUvfQSpPykDjn%2BHfvPXd0d6hU8AYWpdFjEoWMBKBzU0i9H5NtuujdkACd0IVQRsOYHAHJ3WSEolI1nzau1Bt4bQBd4C%2FSyLsfY9JYzY1AiVyLCpC7nCfTxxXvh4wLRLDO48jFUqD0rNK5mlPatm3YDkJj97TPGKPslnpVJ%2FlC3iHqzxm1Nc7kRl0tB9%2FTB4iIsrhDeIUnrC%2FGe6nPI%2FiMkp1FyH%2Bk6bq3%2FpmkJJheu5sZjU0FNgH7JnPYblbnAtBXsC9Ekau8Z0v0k83MIURt2p0SL5Cbqm6cvzX0b3QHBVVmL%2F2slvMxqV9kcbyX24fZYylW7nWe%2Fn42mH0rm6dDNhS%2FtkkyzNrNojXZ91l6k5Pg%2BZpsv0t2idjAUnd5On%2FE%2Bl97LPZPymI%2BiHnEwuEg%2FkOCuc9TauFOzmvRwocdWihK4U61dd%2FUglAV0oeqVuYIsJyv6XX2tMuJnFeFn8h28r8DGNH37svWPfkgPMgomR%2BDhbGVcB%2FnFIXR6UQWvd1JyhuNNeRyJ6HwcSll2pnvwvnddrYBQidn6L%2FbPfkL&RelayState=ver%3A1-hint%

# Chunking 
Splits content into smaller chunks using `RecursiveCharacterTextSplitter` from LangChain.

In [ ]:
# Initialize chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

chunked_docs = text_splitter.split_documents(documents)


# Generate and Store Embeddings

Generates embeddings for text chunks using the `mxbai-embed-large-v1` model from Hugging Face and stores them in an `in-memory Milvus vector DB` for efficient retrieval.

In [ ]:
# Initialize HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

# Store in Milvus
vectorstore = Milvus.from_documents(  
    documents=chunked_docs,
    embedding=embeddings,
    connection_args={"uri": "./milvus_demo.db"},  
    drop_old=True, 
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

print("Successfully stored embeddings in Milvus!")

/opt/anaconda3/envs/Snowflake/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/Snowflake/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Successfully stored embeddings in Milvus!


# Get Models.Corp Credentials

Obtain the API key for Models.Corp by following the instructions at - https://gitlab.cee.redhat.com/models-corp/user-documentation and create a `.env` file at the same location where this notebook is present and insert the the line `ACCESS_TOKEN = "YOUR TOKEN GOES HERE"` in the `.env` file 

As we are going to use **Granite-3.1-8b-instruct** details of the LLM can be found here - https://gitlab.cee.redhat.com/models-corp/user-documentation/-/blob/main/models/granite-3-1-8b-instruct.md?ref_type=heads

In [ ]:
# Load the environment variables from the .env file
load_dotenv()

# Access the access token
access_token = os.getenv("ACCESS_TOKEN")

model_api_url = "https://granite-3-1-8b-instruct--apicast-production.apps.int.stc.ai.prod.us-east-1.aws.paas.redhat.com/v1"
model = "/data/granite-3.1-8b-instruct"

## Query LLM with RAG

This function queries a language model using the `Retrieval-Augmented Generation (RAG)` approach. It retrieves relevant text from Milvus, formats it with a structured prompt, and generates fact-based responses.


In [ ]:
llm = ChatOpenAI(model=model, api_key=access_token, base_url=model_api_url, temperature=0.1)

# Define the prompt template
PROMPT_TEMPLATE = """
Human: You are an AI assistant, and provides answers to questions by using fact based and statistical information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

# Create the prompt template
prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create the chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
query = "How to request a new account?"

res = rag_chain.invoke(query)
print("--------------------------\n")
print("Question : ",query)
print("\n--------------------------\n")
print("Response : ",res)
print("\n--------------------------")



--------------------------

Question :  How to request a new account?

--------------------------

Response :  To request a new account in Red Hat Sales Cloud, follow these steps:

1. Choose the 'Accounts' tab.
2. Hit the 'Search Account' button.
3. Complete the company name field.
4. Fill out as much of this form as possible, including the country.
5. Hit the 'Search' button.
6. If you see the account in the list, click on the name to open the account record.
7. If you need to try another name, hit 'Previous'.
8. If a new account is still needed, choose 'Notify Data Custodian'. The request will be researched, and either a new account will be created based on your data input, or guidance will be given.

This process ensures that the Information Management Team (IMT) can verify the information provided and add more where applicable, ensuring correct customer data and proper placement in the account hierarchy. The IMT uses data from the Dunn & Bradstreet database and the D&B Buydex model

 That looks great. The retriever and the granite model both worked well. 
 

Feel free to try other hosted models on **Models.Corp**. For the list of models on **Models.corp** follow this link - https://gitlab.cee.redhat.com/models-corp/user-documentation/-/tree/main/models?ref_type=heads